# Final PJT - EDA & Preprocessing

이해원

In [2]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns 
import sklearn 

plt.rcParams['font.family'] ='D2Coding'

# pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [3]:
customer = pd.read_csv('SDB2_A_ID.csv')
rlps = pd.read_csv('SDB2_I_CT_RLPS.csv')            # 보험계약관계자정보

cont = pd.read_csv('SDB2_I_CT_CONT_PVT.csv')        # 보험계약정보
mgg = pd.read_csv('SDB2_I_CT_MGG_PVT.csv')          # 보험담보정보

accd = pd.read_csv('SDB2_I_PT_ACCD.csv')            # 청구사고정보 X
claim_cont = pd.read_csv('SDB2_I_PT_CONT.csv')      # 청구계약정보 X

claim_rs = pd.read_csv('SDB2_I_PT_PMNT.csv')            # 청구지급사유정보
claim_rs_dtl = pd.read_csv('SDB2_I_PT_PMNTDTL.csv')      # 청구지급상세사유정보

## 1. EDA

### (1) 기초통계량

In [20]:
df_list = [customer, rlps, cont, mgg, claim_rs, claim_rs_dtl]

for df in df_list:
    print(df.info())
    print(df.describe(include='all'))
    for col in df:
        print(df[col].value_counts())
    print()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49134 entries, 0 to 49133
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype
---  ------       --------------  -----
 0   JOIN_SN      49134 non-null  int64
 1   JOIN_SN_TYP  49134 non-null  int64
 2   BTH_YR       49134 non-null  int64
 3   GENDER       49134 non-null  int64
 4   IS_KRN       49134 non-null  int64
dtypes: int64(5)
memory usage: 1.9 MB
None
            JOIN_SN   JOIN_SN_TYP        BTH_YR        GENDER        IS_KRN
count  4.913400e+04  49134.000000  49134.000000  49134.000000  49134.000000
mean   9.851323e+05      1.034559   1944.108357      1.467965      0.982721
std    5.656103e+05      0.260623    258.690600      0.532483      0.130311
min    8.000000e+00      1.000000      0.000000      0.000000      0.000000
25%    4.938495e+05      1.000000   1960.000000      1.000000      1.000000
50%    9.831750e+05      1.000000   1978.000000      1.000000      1.000000
75%    1.478310e+06      1.000000   1

In [13]:
customer.describe(include='all')

,JOIN_SN,JOIN_SN_TYP,BTH_YR,GENDER,IS_KRN
count,4.913400e+04,49134.000000,49134.000000,49134.000000,49134.000000
mean,9.851323e+05,1.034559,1944.108357,1.467965,0.982721
std,5.656103e+05,0.260623,258.690600,0.532483,0.130311
min,8.000000e+00,1.000000,0.000000,0.000000,0.000000
25%,4.938495e+05,1.000000,1960.000000,1.000000,1.000000
50%,9.831750e+05,1.000000,1978.000000,1.000000,1.000000
75%,1.478310e+06,1.000000,1997.000000,2.000000,1.000000
max,1.959919e+06,3.000000,2010.000000,2.000000,1.000000


In [14]:
id_cate = ['JOIN_SN', 'JOIN_SN_TYP', 'GENDER', 'IS_KRN']
id[id_cate] = id[id_cate].astype('category')

display(id.describe())

display(id.agg(['count', 'size', 'nunique']))

,BTH_YR
count,49134.000000
mean,1944.108357
std,258.690600
min,0.000000
25%,1960.000000
50%,1978.000000
75%,1997.000000
max,2010.000000


,JOIN_SN,JOIN_SN_TYP,BTH_YR,GENDER,IS_KRN
count,49134,49134,49134,49134,49134
size,49134,49134,49134,49134,49134
nunique,49133,2,77,3,2


* JOIN_SN에 중복 1건

In [22]:
id[id.duplicated(subset=['JOIN_SN'], keep=False)]

,JOIN_SN,JOIN_SN_TYP,BTH_YR,GENDER,IS_KRN
21376,248157,1,2008,2,1
44659,248157,3,0,0,0


## 1-2. 보험계약관계자정보

In [23]:
display(rlps.head(2))
display(rlps.info())

,JOIN_SN,JOIN_SN_TYP,SCTR_CD,COM_SN,IS_ME,POL_SN,CT_RLPS_GBN_CD,CT_IN_RLTN_CD,IS_INDIV,GIS_CD
0,1486482,1,6,7623841,0,1771111,C,99,1,34
1,15110,1,8,6937331,0,2197182,C,1,1,11


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 379028 entries, 0 to 379027
Data columns (total 10 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   JOIN_SN         379028 non-null  int64 
 1   JOIN_SN_TYP     379028 non-null  int64 
 2   SCTR_CD         379028 non-null  int64 
 3   COM_SN          379028 non-null  int64 
 4   IS_ME           379028 non-null  int64 
 5   POL_SN          379028 non-null  int64 
 6   CT_RLPS_GBN_CD  379028 non-null  object
 7   CT_IN_RLTN_CD   379028 non-null  int64 
 8   IS_INDIV        379028 non-null  int64 
 9   GIS_CD          379028 non-null  int64 
dtypes: int64(9), object(1)
memory usage: 28.9+ MB


None

In [15]:
rlps.columns

Index(['JOIN_SN', 'JOIN_SN_TYP', 'SCTR_CD', 'COM_SN', 'IS_ME', 'POL_SN',
       'CT_RLPS_GBN_CD', 'CT_IN_RLTN_CD', 'IS_INDIV', 'GIS_CD'],
      dtype='object')

In [24]:
rlps_cate = ['JOIN_SN', 'JOIN_SN_TYP', 'SCTR_CD', 'COM_SN', 'IS_ME', 'POL_SN', 'CT_RLPS_GBN_CD', 'CT_IN_RLTN_CD', 'IS_INDIV', 'GIS_CD']
rlps[rlps_cate] = rlps[rlps_cate].astype('category')

display(rlps.describe())

display(rlps.agg(['count', 'size', 'nunique']))

,JOIN_SN,JOIN_SN_TYP,SCTR_CD,COM_SN,IS_ME,POL_SN,CT_RLPS_GBN_CD,CT_IN_RLTN_CD,IS_INDIV,GIS_CD
count,379028,379028,379028,379028,379028,379028,379028,379028,379028,379028
unique,46050,1,3,148434,1,248753,2,6,2,17
top,30415,1,6,4141313,0,750334,I,1,1,99
freq,60,379028,215888,57,379028,162,229101,247674,378021,97124


,JOIN_SN,JOIN_SN_TYP,SCTR_CD,COM_SN,IS_ME,POL_SN,CT_RLPS_GBN_CD,CT_IN_RLTN_CD,IS_INDIV,GIS_CD
count,379028,379028,379028,379028,379028,379028,379028,379028,379028,379028
size,379028,379028,379028,379028,379028,379028,379028,379028,379028,379028
nunique,46050,1,3,148434,1,248753,2,6,2,17


* CT_RLPS_GBN_CD(계약자/피보험자 구분 코드)에 피보험자(I)인 경우가 많음 (248753 증권 계약 건 중 229101 건이 피보험자)
* 단, 계약자(C)인 건 중에서 CT_IN_RLTN_CD(계-피 관계코드)가 1인 경우는 본인이 피보험자이니 피보험자로 분류해도 괜찮음
* 계-피보험자 관계코드나 거주지역코드 사용 가능 할 지? 어떻게 사용 할 지? 생각 필요

## 1-3. 보험계약정보

In [25]:
display(cont.head(2))
display(cont.info())

,SCTR_CD,POL_SN,IS_GRP_INS,INS_GBN_CD,INS_CT_STAT_CD,CT_CNCLS_DT,CT_TRMNT_DT,CT_ST_DT,CT_END_DT,CT_EFCTV_DT,CT_RST_DT,CT_PY_AMT,CT_PY_CYCLE_CD,CT_PY_PD,IS_CT_DIGNS,CT_JDGMT_CD,CT_CHNL_CD,YM_201806,YM_201807,YM_201808,YM_201809,YM_201810,YM_201811,YM_201812,YM_201901,YM_201902,YM_201903,YM_201904,YM_201905,YM_201906,YM_201907,YM_201908,YM_201909,YM_201910,YM_201911,YM_201912,YM_202001,YM_202002,YM_202003,YM_202004,YM_202005,YM_202006
0,6,3023157,0,3,7,200806,200903,200806,200906,999912,999912,10,2,1,0,1,6,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
1,8,2067576,0,1,2,201012,999912,201012,999912,999912,999912,33,2,20,0,1,3,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 368483 entries, 0 to 368482
Data columns (total 42 columns):
 #   Column          Non-Null Count   Dtype
---  ------          --------------   -----
 0   SCTR_CD         368483 non-null  int64
 1   POL_SN          368483 non-null  int64
 2   IS_GRP_INS      368483 non-null  int64
 3   INS_GBN_CD      368483 non-null  int64
 4   INS_CT_STAT_CD  368483 non-null  int64
 5   CT_CNCLS_DT     368483 non-null  int64
 6   CT_TRMNT_DT     368483 non-null  int64
 7   CT_ST_DT        368483 non-null  int64
 8   CT_END_DT       368483 non-null  int64
 9   CT_EFCTV_DT     368483 non-null  int64
 10  CT_RST_DT       368483 non-null  int64
 11  CT_PY_AMT       368483 non-null  int64
 12  CT_PY_CYCLE_CD  368483 non-null  int64
 13  CT_PY_PD        368483 non-null  int64
 14  IS_CT_DIGNS     368483 non-null  int64
 15  CT_JDGMT_CD     368483 non-null  int64
 16  CT_CHNL_CD      368483 non-null  int64
 17  YM_201806       368483 non-null  int64
 18  YM_2

None

In [26]:
cont.columns

Index(['SCTR_CD', 'POL_SN', 'IS_GRP_INS', 'INS_GBN_CD', 'INS_CT_STAT_CD',
       'CT_CNCLS_DT', 'CT_TRMNT_DT', 'CT_ST_DT', 'CT_END_DT', 'CT_EFCTV_DT',
       'CT_RST_DT', 'CT_PY_AMT', 'CT_PY_CYCLE_CD', 'CT_PY_PD', 'IS_CT_DIGNS',
       'CT_JDGMT_CD', 'CT_CHNL_CD', 'YM_201806', 'YM_201807', 'YM_201808',
       'YM_201809', 'YM_201810', 'YM_201811', 'YM_201812', 'YM_201901',
       'YM_201902', 'YM_201903', 'YM_201904', 'YM_201905', 'YM_201906',
       'YM_201907', 'YM_201908', 'YM_201909', 'YM_201910', 'YM_201911',
       'YM_201912', 'YM_202001', 'YM_202002', 'YM_202003', 'YM_202004',
       'YM_202005', 'YM_202006'],
      dtype='object')

In [27]:
cont_cate = ['SCTR_CD', 'POL_SN', 'IS_GRP_INS', 'INS_GBN_CD', 'INS_CT_STAT_CD',
       'CT_PY_CYCLE_CD', 'IS_CT_DIGNS',
       'CT_JDGMT_CD', 'CT_CHNL_CD', 'YM_201806', 'YM_201807', 'YM_201808',
       'YM_201809', 'YM_201810', 'YM_201811', 'YM_201812', 'YM_201901',
       'YM_201902', 'YM_201903', 'YM_201904', 'YM_201905', 'YM_201906',
       'YM_201907', 'YM_201908', 'YM_201909', 'YM_201910', 'YM_201911',
       'YM_201912', 'YM_202001', 'YM_202002', 'YM_202003', 'YM_202004',
       'YM_202005', 'YM_202006']
cont[cont_cate] = cont[cont_cate].astype('category')

display(cont.describe())

display(cont.agg(['count', 'size', 'nunique']))

,CT_CNCLS_DT,CT_TRMNT_DT,CT_ST_DT,CT_END_DT,CT_EFCTV_DT,CT_RST_DT,CT_PY_AMT,CT_PY_PD
count,368483.000000,368483.000000,368483.000000,368483.000000,368483.000000,368483.000000,3.684830e+05,368483.000000
mean,201374.557407,824804.500148,201377.484584,367966.221088,955179.938477,979490.807391,4.352733e+02,13.975546
std,382.918717,330363.003241,383.483364,322382.183376,183602.393510,126037.479117,8.537547e+03,15.155396
min,200612.000000,199911.000000,200612.000000,200612.000000,199911.000000,200610.000000,1.000000e+00,0.000000
25%,201101.000000,999912.000000,201101.000000,201812.000000,999912.000000,999912.000000,2.800000e+01,1.000000
50%,201405.000000,999912.000000,201406.000000,203404.000000,999912.000000,999912.000000,5.900000e+01,10.000000
75%,201706.000000,999912.000000,201707.000000,208811.000000,999912.000000,999912.000000,1.200000e+02,20.000000
max,202006.000000,999912.000000,202006.000000,999912.000000,999912.000000,999912.000000,1.200000e+06,99.000000


,SCTR_CD,POL_SN,IS_GRP_INS,INS_GBN_CD,INS_CT_STAT_CD,CT_CNCLS_DT,CT_TRMNT_DT,CT_ST_DT,CT_END_DT,CT_EFCTV_DT,CT_RST_DT,CT_PY_AMT,CT_PY_CYCLE_CD,CT_PY_PD,IS_CT_DIGNS,CT_JDGMT_CD,CT_CHNL_CD,YM_201806,YM_201807,YM_201808,YM_201809,YM_201810,YM_201811,YM_201812,YM_201901,YM_201902,YM_201903,YM_201904,YM_201905,YM_201906,YM_201907,YM_201908,YM_201909,YM_201910,YM_201911,YM_201912,YM_202001,YM_202002,YM_202003,YM_202004,YM_202005,YM_202006
count,368483,368483,368483,368483,368483,368483,368483,368483,368483,368483,368483,368483,368483,368483,368483,368483,368483,368483,368483,368483,368483,368483,368483,368483,368483,368483,368483,368483,368483,368483,368483,368483,368483,368483,368483,368483,368483,368483,368483,368483,368483,368483
size,368483,368483,368483,368483,368483,368483,368483,368483,368483,368483,368483,368483,368483,368483,368483,368483,368483,368483,368483,368483,368483,368483,368483,368483,368483,368483,368483,368483,368483,368483,368483,368483,368483,368483,368483,368483,368483,368483,368483,368483,368483,368483
nunique,3,244232,2,18,10,163,181,163,1120,174,166,233,4,100,2,8,8,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2


* IS_GRP_INS(단체보험여부) : 단체보험인 경우는 개인 의지로 가입한 상품이 아니니 제외?
  * 이미 단체보험이 있으니 개인 보험이 필요 없어서 가입하지 않는 경우
  * 단체보험이 있어도 보험에 대한 무지 혹은 필요성으로 추가로 개인이 보험 가입하는 경우
  
  ... 등 이 있을 수도?

* 주 계약상태코드 / 보험계약체결월, 보험계약해지월 / 기준년월 등 으로 계약유지율 비슷한 내용 뽑을 수 있을지? (뽑는다면 보험종류 별이 좋지 않을까)
* 납입보험료로 고객이 보험에 지출한 총액(보험종류 별? 보험 전체?)
* 가입경로코드 피쳐에 넣어야?

## 1-4. 보험담보정보

In [32]:
display(mgg.head(2))
display(mgg.info())

,JOIN_SN,JOIN_SN_TYP,SCTR_CD,POL_SN,MGG_GRP_CD,MGG_CD,MGG_CNT,MGG_AMT_SUM,MGG_AMT_MAX,MGG_AMT_MIN,YM_201806,YM_201807,YM_201808,YM_201809,YM_201810,YM_201811,YM_201812,YM_201901,YM_201902,YM_201903,YM_201904,YM_201905,YM_201906,YM_201907,YM_201908,YM_201909,YM_201910,YM_201911,YM_201912,YM_202001,YM_202002,YM_202003,YM_202004,YM_202005,YM_202006
0,499723,1,6,3260523,A5,A5301,1,480,480,480,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
1,998541,1,6,1272317,A4,A4303,1,400,400,400,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1408807 entries, 0 to 1408806
Data columns (total 35 columns):
 #   Column       Non-Null Count    Dtype 
---  ------       --------------    ----- 
 0   JOIN_SN      1408807 non-null  int64 
 1   JOIN_SN_TYP  1408807 non-null  int64 
 2   SCTR_CD      1408807 non-null  int64 
 3   POL_SN       1408807 non-null  int64 
 4   MGG_GRP_CD   1408807 non-null  object
 5   MGG_CD       1408807 non-null  object
 6   MGG_CNT      1408807 non-null  int64 
 7   MGG_AMT_SUM  1408807 non-null  int64 
 8   MGG_AMT_MAX  1408807 non-null  int64 
 9   MGG_AMT_MIN  1408807 non-null  int64 
 10  YM_201806    1408807 non-null  int64 
 11  YM_201807    1408807 non-null  int64 
 12  YM_201808    1408807 non-null  int64 
 13  YM_201809    1408807 non-null  int64 
 14  YM_201810    1408807 non-null  int64 
 15  YM_201811    1408807 non-null  int64 
 16  YM_201812    1408807 non-null  int64 
 17  YM_201901    1408807 non-null  int64 
 18  YM_201902    1408807 n

None

In [33]:
mgg.columns

Index(['JOIN_SN', 'JOIN_SN_TYP', 'SCTR_CD', 'POL_SN', 'MGG_GRP_CD', 'MGG_CD',
       'MGG_CNT', 'MGG_AMT_SUM', 'MGG_AMT_MAX', 'MGG_AMT_MIN', 'YM_201806',
       'YM_201807', 'YM_201808', 'YM_201809', 'YM_201810', 'YM_201811',
       'YM_201812', 'YM_201901', 'YM_201902', 'YM_201903', 'YM_201904',
       'YM_201905', 'YM_201906', 'YM_201907', 'YM_201908', 'YM_201909',
       'YM_201910', 'YM_201911', 'YM_201912', 'YM_202001', 'YM_202002',
       'YM_202003', 'YM_202004', 'YM_202005', 'YM_202006'],
      dtype='object')

In [34]:
mgg_cate = ['JOIN_SN', 'JOIN_SN_TYP', 'SCTR_CD', 'POL_SN', 'MGG_GRP_CD', 'MGG_CD', 'YM_201806',
       'YM_201807', 'YM_201808', 'YM_201809', 'YM_201810', 'YM_201811',
       'YM_201812', 'YM_201901', 'YM_201902', 'YM_201903', 'YM_201904',
       'YM_201905', 'YM_201906', 'YM_201907', 'YM_201908', 'YM_201909',
       'YM_201910', 'YM_201911', 'YM_201912', 'YM_202001', 'YM_202002',
       'YM_202003', 'YM_202004', 'YM_202005', 'YM_202006']
mgg[mgg_cate] = mgg[mgg_cate].astype('category')

display(mgg.describe())

display(mgg.agg(['count', 'size', 'nunique']))

,MGG_CNT,MGG_AMT_SUM,MGG_AMT_MAX,MGG_AMT_MIN
count,1.408807e+06,1.408807e+06,1.408807e+06,1.408807e+06
mean,1.320381e+00,1.495291e+04,1.335901e+04,1.187163e+04
std,1.299050e+00,3.154855e+04,2.868541e+04,2.731272e+04
min,1.000000e+00,1.000000e+02,1.000000e+02,1.000000e+02
25%,1.000000e+00,2.600000e+02,2.400000e+02,2.100000e+02
50%,1.000000e+00,5.400000e+03,4.500000e+03,2.600000e+03
75%,1.000000e+00,1.300000e+04,1.200000e+04,1.000000e+04
max,3.400000e+01,5.000000e+05,5.000000e+05,5.000000e+05


,JOIN_SN,JOIN_SN_TYP,SCTR_CD,POL_SN,MGG_GRP_CD,MGG_CD,MGG_CNT,MGG_AMT_SUM,MGG_AMT_MAX,MGG_AMT_MIN,YM_201806,YM_201807,YM_201808,YM_201809,YM_201810,YM_201811,YM_201812,YM_201901,YM_201902,YM_201903,YM_201904,YM_201905,YM_201906,YM_201907,YM_201908,YM_201909,YM_201910,YM_201911,YM_201912,YM_202001,YM_202002,YM_202003,YM_202004,YM_202005,YM_202006
count,1408807,1408807,1408807,1408807,1408807,1408807,1408807,1408807,1408807,1408807,1408807,1408807,1408807,1408807,1408807,1408807,1408807,1408807,1408807,1408807,1408807,1408807,1408807,1408807,1408807,1408807,1408807,1408807,1408807,1408807,1408807,1408807,1408807,1408807,1408807
size,1408807,1408807,1408807,1408807,1408807,1408807,1408807,1408807,1408807,1408807,1408807,1408807,1408807,1408807,1408807,1408807,1408807,1408807,1408807,1408807,1408807,1408807,1408807,1408807,1408807,1408807,1408807,1408807,1408807,1408807,1408807,1408807,1408807,1408807,1408807
nunique,50143,1,3,115467,13,140,31,311,310,311,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2


* 이 데이터 어떻게 씀?
* 군집별 선호하는 담보?
* 담보별 손해율 계산 불가
* 보험사에서 다 짜놓은 담보 중에서 필수로 들어가는 담보는 당연히 빈도가 높을 수 밖에 없지 않나?

## 1-5. 청구사고정보

In [35]:
display(accd.head(2))
display(accd.info())

,JOIN_SN,JOIN_SN_TYP,SCTR_CD,COM_SN,IS_ME,ACCD_SN,REQ_SN,IS_INDIV,GIS_CD,ACCD_DT,FST_RPT_DT,ACCD_GIS_CD,IN_DRV_RLTN_CD
0,514694,1,8,5561038,0,2292889,839524,1,99,201905,201905,99,0
1,1666172,1,8,1604841,0,598965,1898453,1,36,201902,201905,99,0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41347 entries, 0 to 41346
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype
---  ------          --------------  -----
 0   JOIN_SN         41347 non-null  int64
 1   JOIN_SN_TYP     41347 non-null  int64
 2   SCTR_CD         41347 non-null  int64
 3   COM_SN          41347 non-null  int64
 4   IS_ME           41347 non-null  int64
 5   ACCD_SN         41347 non-null  int64
 6   REQ_SN          41347 non-null  int64
 7   IS_INDIV        41347 non-null  int64
 8   GIS_CD          41347 non-null  int64
 9   ACCD_DT         41347 non-null  int64
 10  FST_RPT_DT      41347 non-null  int64
 11  ACCD_GIS_CD     41347 non-null  int64
 12  IN_DRV_RLTN_CD  41347 non-null  int64
dtypes: int64(13)
memory usage: 4.1 MB


None

In [36]:
accd.columns

Index(['JOIN_SN', 'JOIN_SN_TYP', 'SCTR_CD', 'COM_SN', 'IS_ME', 'ACCD_SN',
       'REQ_SN', 'IS_INDIV', 'GIS_CD', 'ACCD_DT', 'FST_RPT_DT', 'ACCD_GIS_CD',
       'IN_DRV_RLTN_CD'],
      dtype='object')

In [37]:
accd_cate = ['JOIN_SN', 'JOIN_SN_TYP', 'SCTR_CD', 'COM_SN', 'IS_ME', 'ACCD_SN',
       'REQ_SN', 'IS_INDIV', 'GIS_CD', 'ACCD_GIS_CD', 'IN_DRV_RLTN_CD']
accd[accd_cate] = accd[accd_cate].astype('category')

display(accd.describe())

display(accd.agg(['count', 'size', 'nunique']))

,ACCD_DT,FST_RPT_DT
count,41347.000000,41347.000000
mean,201894.560089,201913.396159
std,65.656164,66.722906
min,201806.000000,201806.000000
25%,201812.000000,201902.000000
50%,201905.000000,201908.000000
75%,201911.000000,202001.000000
max,202006.000000,202006.000000


,JOIN_SN,JOIN_SN_TYP,SCTR_CD,COM_SN,IS_ME,ACCD_SN,REQ_SN,IS_INDIV,GIS_CD,ACCD_DT,FST_RPT_DT,ACCD_GIS_CD,IN_DRV_RLTN_CD
count,41347,41347,41347,41347,41347,41347,41347,41347,41347,41347,41347,41347,41347
size,41347,41347,41347,41347,41347,41347,41347,41347,41347,41347,41347,41347,41347
nunique,17591,1,3,22189,1,39650,33603,1,17,25,25,17,6


* 거주지역 코드는 보험계약관계자정보에도 있음.
* 사고지역 코드 쓸 필요가 있을까
* 차량 사고에만 관한 데이터인지 확인 필요
  * 청구계약정보 기준, 자동차보험이 아니면서 청구사고정보에 있는 데이터 존재함
  
  => 차량 사고에만 관한 데이터 아님

* 사고번호로만 조회 했을 때, 같은 번호로 중복된 데이터 중 차주번호, 청구번호가 다른 경우 확인

  => 청구사고정보, 청구계약번호 결합 시, 사고번호와 청구번호가 일치하는 것으로 데이터 결합해야 함

## 1-6. 청구계약정보

In [38]:
display(claim_cont.head(2))
display(claim_cont.info())

,SCTR_CD,ACCD_SN,REQ_SN,POL_SN,IS_GRP_INS,INS_GBN_CD,CT_CNCLS_DT,CT_ST_DT,CT_END_DT,CT_IN_RLTN_CD,CT_CHNL_CD
0,6,1140150,650855,5957565,0,4,201601,201601,207501,99,99
1,6,1935093,1630531,5097060,0,3,201107,201107,208007,1,1


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46299 entries, 0 to 46298
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype
---  ------         --------------  -----
 0   SCTR_CD        46299 non-null  int64
 1   ACCD_SN        46299 non-null  int64
 2   REQ_SN         46299 non-null  int64
 3   POL_SN         46299 non-null  int64
 4   IS_GRP_INS     46299 non-null  int64
 5   INS_GBN_CD     46299 non-null  int64
 6   CT_CNCLS_DT    46299 non-null  int64
 7   CT_ST_DT       46299 non-null  int64
 8   CT_END_DT      46299 non-null  int64
 9   CT_IN_RLTN_CD  46299 non-null  int64
 10  CT_CHNL_CD     46299 non-null  int64
dtypes: int64(11)
memory usage: 3.9 MB


None

In [39]:
claim_cont.columns

Index(['SCTR_CD', 'ACCD_SN', 'REQ_SN', 'POL_SN', 'IS_GRP_INS', 'INS_GBN_CD',
       'CT_CNCLS_DT', 'CT_ST_DT', 'CT_END_DT', 'CT_IN_RLTN_CD', 'CT_CHNL_CD'],
      dtype='object')

In [40]:
claim_cont_cate = ['SCTR_CD', 'ACCD_SN', 'REQ_SN', 'POL_SN', 'IS_GRP_INS', 'INS_GBN_CD', 'CT_IN_RLTN_CD', 'CT_CHNL_CD']
claim_cont[claim_cont_cate] = claim_cont[claim_cont_cate].astype('category')

display(claim_cont.describe())

display(claim_cont.agg(['count', 'size', 'nunique']))

,CT_CNCLS_DT,CT_ST_DT,CT_END_DT
count,46299.000000,46299.000000,46299.000000
mean,201320.236873,201320.941791,379891.560638
std,466.796419,467.399438,329693.075278
min,199911.000000,199911.000000,201803.000000
25%,201002.000000,201002.000000,202505.000000
50%,201401.000000,201401.000000,205602.000000
75%,201801.000000,201801.000000,209402.000000
max,202009.000000,202009.000000,999912.000000


,SCTR_CD,ACCD_SN,REQ_SN,POL_SN,IS_GRP_INS,INS_GBN_CD,CT_CNCLS_DT,CT_ST_DT,CT_END_DT,CT_IN_RLTN_CD,CT_CHNL_CD
count,46299,46299,46299,46299,46299,46299,46299,46299,46299,46299,46299
size,46299,46299,46299,46299,46299,46299,46299,46299,46299,46299,46299
nunique,3,39616,33570,26331,2,19,251,251,983,6,8


* 사고 발생월에 관한 데이터가 없어서 보험계약 체결 시점부터 사고발생까지 걸린 시간 계산 불가
* 어떤 정보가 쓸만한지 눈에 확 와닿는건 없음


## 1-7. 청구지급사유정보

In [41]:
display(pmnt.head(2))
display(pmnt.info())

,ACCD_SN,REQ_SN,SCTR_CD,POL_SN,INS_GBN_CD,VCTM_SN,PMNT_RESN_CD,PMNT_DT,IS_PMNT_INVST,PMNT_JDGMNT_CD
0,1433170,1818958,6,9641244,4,2547,B2005,202005,0,1
1,797449,671787,6,5047760,17,260,C1008,201808,1,11


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 67799 entries, 0 to 67798
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   ACCD_SN         67799 non-null  int64 
 1   REQ_SN          67799 non-null  int64 
 2   SCTR_CD         67799 non-null  int64 
 3   POL_SN          67799 non-null  int64 
 4   INS_GBN_CD      67799 non-null  int64 
 5   VCTM_SN         67799 non-null  int64 
 6   PMNT_RESN_CD    67799 non-null  object
 7   PMNT_DT         67799 non-null  int64 
 8   IS_PMNT_INVST   67799 non-null  int64 
 9   PMNT_JDGMNT_CD  67799 non-null  int64 
dtypes: int64(9), object(1)
memory usage: 5.2+ MB


None

In [42]:
pmnt.columns

Index(['ACCD_SN', 'REQ_SN', 'SCTR_CD', 'POL_SN', 'INS_GBN_CD', 'VCTM_SN',
       'PMNT_RESN_CD', 'PMNT_DT', 'IS_PMNT_INVST', 'PMNT_JDGMNT_CD'],
      dtype='object')

In [44]:
pmnt_cate = ['ACCD_SN', 'REQ_SN', 'SCTR_CD', 'POL_SN', 'INS_GBN_CD', 'VCTM_SN',
       'PMNT_RESN_CD', 'IS_PMNT_INVST', 'PMNT_JDGMNT_CD']
pmnt[pmnt_cate] = pmnt[pmnt_cate].astype('category')

display(pmnt.describe())

display(pmnt.agg(['count', 'size', 'nunique']))

,PMNT_DT
count,67799.000000
mean,201915.369681
std,66.284414
min,201806.000000
25%,201902.000000
50%,201908.000000
75%,202001.000000
max,202006.000000


,ACCD_SN,REQ_SN,SCTR_CD,POL_SN,INS_GBN_CD,VCTM_SN,PMNT_RESN_CD,PMNT_DT,IS_PMNT_INVST,PMNT_JDGMNT_CD
count,67799,67799,67799,67799,67799,67799,67799,67799,67799,67799
size,67799,67799,67799,67799,67799,67799,67799,67799,67799,67799
nunique,38742,32798,3,25277,19,368,50,25,2,8


* 청구지급상세사유정보와 데이터가 일치하지 않는 것으로 보임
  * 좀 더 확실하려면 테이블 간 일치 정도를 확인해봐야...
* 청구지급사유정보 보다는 청구지급상세사유정보가 더 유용해 보임

## 1-8. 청구지급상세사유정보

In [45]:
display(pmntdtl.head(2))
display(pmntdtl.info())

,ACCD_SN,REQ_SN,SCTR_CD,POL_SN,VCTM_SN,PMNT_RESN_CD,PMNT_DT,PMNT_RESN_DTL_CD,PMNT_DTL_SN,PMNT_AMT
0,1132050,953774,8,8482978,1172,A1001,201809,1,1,10000
1,2141286,76910,6,3612735,23,B2001,202001,1,1,9400


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 71249 entries, 0 to 71248
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   ACCD_SN           71249 non-null  int64 
 1   REQ_SN            71249 non-null  int64 
 2   SCTR_CD           71249 non-null  int64 
 3   POL_SN            71249 non-null  int64 
 4   VCTM_SN           71249 non-null  int64 
 5   PMNT_RESN_CD      71249 non-null  object
 6   PMNT_DT           71249 non-null  int64 
 7   PMNT_RESN_DTL_CD  71249 non-null  int64 
 8   PMNT_DTL_SN       71249 non-null  int64 
 9   PMNT_AMT          71249 non-null  int64 
dtypes: int64(9), object(1)
memory usage: 5.4+ MB


None

In [46]:
pmntdtl.columns

Index(['ACCD_SN', 'REQ_SN', 'SCTR_CD', 'POL_SN', 'VCTM_SN', 'PMNT_RESN_CD',
       'PMNT_DT', 'PMNT_RESN_DTL_CD', 'PMNT_DTL_SN', 'PMNT_AMT'],
      dtype='object')

In [47]:
pmntdtl_cate = ['ACCD_SN', 'REQ_SN', 'SCTR_CD', 'POL_SN', 'VCTM_SN', 'PMNT_RESN_CD',
       'PMNT_RESN_DTL_CD', 'PMNT_DTL_SN']
pmntdtl[pmntdtl_cate] = pmntdtl[pmntdtl_cate].astype('category')

display(pmntdtl.describe())

display(pmntdtl.agg(['count', 'size', 'nunique']))

,PMNT_DT,PMNT_AMT
count,71249.000000,71249.000000
mean,201915.905079,386.261898
std,65.997586,1281.555500
min,201806.000000,0.000000
25%,201902.000000,32.000000
50%,201908.000000,110.000000
75%,202001.000000,320.000000
max,202006.000000,50000.000000


,ACCD_SN,REQ_SN,SCTR_CD,POL_SN,VCTM_SN,PMNT_RESN_CD,PMNT_DT,PMNT_RESN_DTL_CD,PMNT_DTL_SN,PMNT_AMT
count,71249,71249,71249,71249,71249,71249,71249,71249,71249,71249
size,71249,71249,71249,71249,71249,71249,71249,71249,71249,71249
nunique,37890,31945,3,24719,356,50,25,12,13,273


* 보험지금금액, 사고-청구번호로 보험금 수령 빈도, 보험금 확인 가능하지 않을까


## 1-9. 청구피해자물정보

In [ ]:
display(vctm.head(2))
display(vctm.info())

In [ ]:
vctm.columns

In [ ]:
vctm_cate = []
vctm[vctm_cate] = vctm[vctm_cate].astype('category')

display(vctm.describe())

display(vctm.agg(['count', 'size', 'nunique']))

* 어따 써

## 1-10. 내 생각 더 추가

* 세세한 정보로 갈 수록 차주ID, 증권번호, 사고번호와 같은 고유 값에 대한 건수가 줄어듬

    (차주정보->보험계약관계자정보->보험계약정보 / 차주정보->청구사고정보->청구계약정보->청구지급사유정보->청구지급상세사유정보)
* 테이블 간 결합 시, 데이터의 개수가 줄어들 것으로 예상
* 하위 테이블에는 데이터가 있는데 상위 테이블에는 데이터가 없는 경우가 있는 지 확인 필요
* 연체정보, 개인대출정보, 신용카드정보, 보험정보가 있는 고객을 보험 정보로만 군집 분석 한 다음에

 군집별 연체정보, 개인대출정보, 신용카드정보 특성이 나온다면

 보험정보가 없는 고객 대상으로 고객 분류 모델링 돌려서 보험 군집에 넣어보고 유사 군집의 선호하는 보험 종류를 추천할 수 있을까?

* 모든 데이터를 군집 분석 변수에 넣지 않더라도 특성으로 결과가 나올 수도 있으니 한 행으로 만들어야...?
  * 에바 아닐까

# 2.테이블 간 데이터 일치 정도?

In [28]:
# id 테이블 중, 개인 차주일련번호 unique 값
id_type1 = id[id['JOIN_SN_TYP'] == 1]
id_unique = pd.DataFrame(id_type1['JOIN_SN'].unique(), columns=['JOIN_SN'])
id_unique

,JOIN_SN
0,787068
1,1630591
2,746631
3,936890
4,1850766
...,...
48280,158999
48281,1731608
48282,656440
48283,285890


In [31]:
# rlps 보험계약관계자정보 중, 개인 차주일련번호, 증권번호 unique 값
rlps_type1 = rlps[rlps['JOIN_SN_TYP'] == 1]
rlps_unique = rlps_type1[['JOIN_SN', 'POL_SN']].drop_duplicates().sort_values(by='JOIN_SN')
rlps_unique

,JOIN_SN,POL_SN
253614,8,3700186
117086,8,9074314
203593,8,6261020
176303,8,7699415
202174,8,6394418
...,...,...
140933,1959919,5854667
35746,1959919,419312
179181,1959919,5003735
76175,1959919,3831658
